# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

# Import Dataset

In [9]:
!pip install kaggle, kagglehub

ERROR: Invalid requirement: 'kaggle,': Expected end or semicolon (after name and no valid version specifier)
    kaggle,
          ^


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

The syntax of the command is incorrect.


'cp' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
import kagglehub, kaggle

In [13]:
# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")
print("Path to dataset files:", path)

100%|██████████| 1.06G/1.06G [03:00<00:00, 6.34MB/s]

Extracting files...


Path to dataset files: C:\Users\7200\.cache\kagglehub\datasets\salader\dogs-vs-cats\versions\1


# VGG16 Model Transfer Learning

In [14]:
from tensorflow.keras.applications.vgg16 import VGG16

In [32]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top=False,
    input_shape=(150, 150, 3),
    name='conv_base',
)
conv_base.trainable = False
conv_base.summary()

Model: "conv_base"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

# Transfer Learning Model

In [ ]:
model = Sequential()

## Add convolutional base
model.add(conv_base)

## Add Fully Connected Layers
model.add(Flatten(name='flatten_layer'))

## Hidden layers
model.add(Dense(units=256, activation='relu', name='fc_layer_1'))

## Output layers
model.add(Dense(units=1, activation='sigmoid', name='output_layer'))

In [ ]:
model.name = 'vgg16_transfer_learned'
model.summary()

Model: "vgg16_transfer_learned"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv_base (Functional)          │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer (Flatten)         │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc_layer_1 (Dense)              │ (None, 256)            │     2,097,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,812,353 (64.13 MB)

 Trainable params: 2,097,665 (8.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

# Prepare data for model training

In [ ]:
from keras.utils import image_dataset_from_directory

In [ ]:
# generators
train_data = image_dataset_from_directory(
    directory = 'Cat_Dog_Dataset/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

validation_data = image_dataset_from_directory(
    directory = 'Cat_Dog_Dataset/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(150,150)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [38]:
## Normalize dataset for training

def preprocess(image, label):
    image = tensorflow.cast(image/255.0, tensorflow.float32)
    return image, label

train_data = train_data.map(preprocess)
validation_data = validation_data.map(preprocess)

# Start Training

In [40]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_data, epochs=10, validation_data=validation_data).history

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2509s 4s/step - accuracy: 0.8491 - loss: 0.3679 - val_accuracy: 0.9100 - val_loss: 0.2113
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2515s 4s/step - accuracy: 0.9125 - loss: 0.2056 - val_accuracy: 0.9158 - val_loss: 0.2048
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2744s 4s/step - accuracy: 0.9327 - loss: 0.1669 - val_accuracy: 0.9106 - val_loss: 0.2244
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2957s 5s/step - accuracy: 0.9449 - loss: 0.1372 - val_accuracy: 0.9160 - val_loss: 0.2224
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 2270s 4s/step - accuracy: 0.9547 - loss: 0.1137 - val_accuracy: 0.9170 - val_loss: 0.2402
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.9658 - loss: 0.0913 

KeyboardInterrupt: 

In [ ]:
history = model.fit(train_data, epochs=10, validation_data=validation_data, initial_epoch=6).history

Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6454s 10s/step - accuracy: 0.9758 - loss: 0.0652 - val_accuracy: 0.8906 - val_loss: 0.3732
Epoch 8/10
562/625 ━━━━━━━━━━━━━━━━━━━━ 4:14 4s/step - accuracy: 0.9770 - loss: 0.0608

# Plot Learning Curve

In [ ]:
plt.plot(history['loss'], label='train loss', color='red')
plt.plot(history['val_loss'], label='val loss', color='green')
plt.legend()
plt.show()
plt.savefig('Feature_Extraction_Without_Data_Aug/loss.png')

plt.plot(history['accuracy'], label='train accuracy', color='red')
plt.plot(history['val_accuracy'], label='val accuracy', color='green')
plt.legend()
plt.show()
plt.savefig('Feature_Extraction_Without_Data_Aug/accuracy.png')

# Save Model

In [ ]:
model.save('Feature_Extraction_Without_Data_Aug/cat_dog_cls_vgg16_transfer_learning.keras')